In [1]:
!pip install openai-agents -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00


In [2]:
%cd /content/drive/MyDrive/AI Agents/OpenAI Agents SDK/AI Sommelier

/content/drive/MyDrive/AI Agents/OpenAI Agents SDK/AI Sommelier


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [31]:
import asyncio
from uuid import uuid4

from openai import OpenAI
from agents import (
    Agent,
    Runner,
    trace,
    ItemHelpers,
    MessageOutputItem,
    HandoffOutputItem,
    ToolCallOutputItem,
    FileSearchTool,
    function_tool,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

client = OpenAI()

# Upload the file to OpenAI

In [ ]:
# Upload a file once, reuse everywhere
path = "Vinhos baba d'urso.pdf"
upload = client.files.create(
    file=open(path, "rb"),
    purpose="assistants",
)

In [ ]:
# Retrieve and print the file ID
FILE_ID = upload.id
FILE_ID

'file-8GWrzJBcLBBuR8XktaP5fv'

In [ ]:
# Create the vector store with code
vector_store = client.vector_stores.create(
    name = "Vector-Store-X"
)

In [ ]:
# Insert the file in the vector store
vs_file = client.vector_stores.files.create(
    vector_store_id = 'vs_683efab3660c8191ba3e4df02943ed51',
    file_id=FILE_ID,
)

# AI Sommelier Agent

In [5]:
# Define the instructions and other configs
instructions = f"""{RECOMMENDED_PROMPT_PREFIX}
You are Bitte's AI Sommelier:
Your task is to provide the user with a wine recommendation for their dish.
Use FileSearchTool to pick up to 3 wines from the vector store.
If the interaction has nothing to do with the wine, do the handoff.
"""
vs_id = "vs_683f09129dcc81919305e1510f0c9da4"

In [28]:
# Define the escalate to boss function
@function_tool(
    name_override = "escalate_to_boss",
    description_override="Talk to the guest about wine. If the topic is not wine, escalate to the boss"
)

async def escalate_to_boss(request: str) -> str:
  print(f"Escalating to the boss {request}")
  return "The boss will be here soon"

In [29]:
# Build the AI Agent
sommelier = Agent(
    name = "Bitte AI Sommelier",
    model = "gpt-4.1-mini",
    instructions = instructions,
    tools = [
        FileSearchTool(vector_store_ids = [vs_id],
                       max_num_results = 3),
        escalate_to_boss
    ]
)

In [34]:
# Build the Chat interactions
async def chat():
  # Store the conversation
  history = []

  # while loop for the conversation
  while True:
    user = input("User: ")
    if user.lower() in {"exit", "quit"}:
      break

    history.append({"role": "user", "content": user})

    # Apply tracing to the conversation
    with trace("Bitte Sommelier", group_id = uuid4().hex):
      run = await Runner.run(sommelier, history)

    stop_chat = False

    for itm in run.new_items:
      if isinstance(itm, MessageOutputItem):
        print("Sommelier", ItemHelpers.text_message_output(itm))

      elif isinstance(itm, ToolCallOutputItem):
        print("Boss was summoned")
        stop_chat = True
        break

    if stop_chat:
      break

    history = run.to_input_list()

# Run the chat
await chat()

User: I would like to pair wine with a seafood rice
Sommelier For a seafood rice dish, a white wine with good acidity and freshness typically pairs well to complement the flavors without overpowering the seafood.

From the wines available, I recommend these options:

1. Branco Alentejo Adega Mayor Caiado or Reserva: These are white wines made from Arinto and Fernão Pires grapes, which tend to have bright acidity and citrus notes that would enhance the seafood flavors in the rice.

2. Branco Algarve Villa Alvor Sauvignon Blanc: Sauvignon Blanc usually offers a zesty acidity and herbaceous notes, which can be refreshing with a seafood rice dish.

3. Branco Espanha Rosa Ruiz Albariño: Albariño is known for its crisp acidity and stone fruit flavors, making it a classic match for seafood dishes.

Any of these white wines would be excellent choices to pair with your seafood rice, bringing balance and enhancing the dish's flavors without overwhelming it. Would you like more details about any 

# What would you do to improve?
If you submit an improvement, I will give you personalized feedback